In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

print("P loaded")

P loaded


In [3]:
_data_dir = './mnist'
mnist = input_data.read_data_sets(_data_dir, one_hot=True)
img_test = mnist.test.images
label_test = mnist.test.labels

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [4]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
NUM_HIDDEN_NODE = 256

x = tf.placeholder("float", [None, IMAGE_PIXELS])
y = tf.placeholder("float", [None, NUM_CLASSES])

W1 = tf.Variable(tf.random_normal([IMAGE_PIXELS, NUM_HIDDEN_NODE], stddev=0.1))
b1 = tf.Variable(tf.zeros([NUM_HIDDEN_NODE]))

W2 = tf.Variable(tf.random_normal([NUM_HIDDEN_NODE, NUM_CLASSES], stddev=0.1))
b2 = tf.Variable(tf.zeros([NUM_CLASSES]))

In [8]:
def inference_mlp(data_in, weight_1, bias_1, weight_2, bias_2):
    hypothesis_1 = tf.matmul(data_in, weight_1) + bias_1
    gz_1 = tf.sigmoid(hypothesis_1)
    hypothesis_2 = tf.matmul(gz_1, weight_2) + bias_2
    return hypothesis_2

def cost(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    return train

def evaluation(logits, labels):
    pred = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(labels, 1))
    accr = tf.reduce_mean(tf.cast(pred, "float"))
    return accr

In [9]:
learning_rate = 0.001

logits = inference_mlp(x, W1, b1, W2, b2)
loss = cost(logits, y)
train = training(loss, learning_rate)
eval_correct = evaluation(logits, y)

init = tf.initialize_all_variables()

In [13]:
n_samples = mnist.train.num_examples
batch_size = 200
total_batch = int(n_samples / batch_size)
total_epoch = 100

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(total_epoch):
        avg_cost = 0.
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed = {x: batch_xs, y: batch_ys}
            sess.run(train, feed)
            avg_cost += sess.run(loss, feed) / batch_size
            
        if epoch % 5 == 0:
            feeds_train = {x: batch_xs, y: batch_ys}
            feeds_test = {x: img_test, y: label_test}
            train_acc = sess.run(eval_correct, feeds_train)
            test_acc = sess.run(eval_correct, feeds_test)
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f"
                  % (epoch, total_epoch, avg_cost, train_acc, test_acc))
            
print ("DONE")

Epoch: 000/100 cost: 0.811026587 train_acc: 0.915 test_acc: 0.914
Epoch: 005/100 cost: 0.204492467 train_acc: 0.955 test_acc: 0.957
Epoch: 010/100 cost: 0.110938252 train_acc: 0.980 test_acc: 0.970
Epoch: 015/100 cost: 0.064654506 train_acc: 0.990 test_acc: 0.976
Epoch: 020/100 cost: 0.037795106 train_acc: 0.990 test_acc: 0.979
Epoch: 025/100 cost: 0.021523071 train_acc: 0.995 test_acc: 0.980
Epoch: 030/100 cost: 0.011925845 train_acc: 1.000 test_acc: 0.981
Epoch: 035/100 cost: 0.006695282 train_acc: 1.000 test_acc: 0.981
Epoch: 040/100 cost: 0.003729988 train_acc: 1.000 test_acc: 0.982
Epoch: 045/100 cost: 0.002065380 train_acc: 1.000 test_acc: 0.982
Epoch: 050/100 cost: 0.001158875 train_acc: 1.000 test_acc: 0.982
Epoch: 055/100 cost: 0.000637483 train_acc: 1.000 test_acc: 0.982
Epoch: 060/100 cost: 0.000345189 train_acc: 1.000 test_acc: 0.982
Epoch: 065/100 cost: 0.000197429 train_acc: 1.000 test_acc: 0.982
Epoch: 070/100 cost: 0.000105296 train_acc: 1.000 test_acc: 0.982
Epoch: 075